# **Case de Análise de Dados em um Marketplace de Eletrônicos**

### **Objetivo**

##### Esta análise tem como objetivo compreender melhor o desempenho de um marketplace de eletrônicos, avaliando indicadores financeiros e operacionais para embasar decisões estratégicas. Através da exploração e tratamento de dados, buscamos identificar padrões de faturamento, lucro, comportamento dos clientes e eficiência das operações.
##### Para isso, utilizamos consultas SQL para extrair e processar informações a partir de três bases de dados principais:
##### • **Pedidos**  – Registros de todas as vendas realizadas.
##### • **Produtos** – Informações sobre os produtos disponíveis na plataforma.
##### • **Clientes** – Detalhes sobre os clientes cadastrados.
##### Posteriormente, os resultados foram visualizados no Power BI para facilitar a interpretação dessas informações.

### **Etapas do Projeto**
##### **A. Primeiro Passo: Carregamento e processamento os dados.**
##### Nesta etapa, os arquivos CSV contendo informações de pedidos, produtos e clientes serão importados e transformados em DataFrames para facilitar a análise. Além disso, será feita uma verificação inicial para garantir a integridade dos dados, identificando e tratando possíveis valores ausentes ou inconsistências.
##### **B. Segundo Passo: Armazenamento temporário em banco de dados.**
##### Para otimizar a manipulação dos dados, será criado um banco de dados SQLite em memória, onde as tabelas serão armazenadas temporariamente. Isso permitirá a execução de consultas SQL mais eficientes, facilitando o cruzamento de informações entre pedidos, produtos e clientes.
#####  **C. Terceiro Passo: Análise Exploratória dos Dados (EDA) para responder às perguntas de negócio.**
##### A análise exploratória buscará identificar padrões de compra e tendências no marketplace. Serão analisados o faturamento, lucro mensal e volume de vendas, além da sazonalidade e a lucratividade dos produtos mais vendidos. Também serão investigados os métodos de pagamento mais utilizados, a taxa de cancelamento e a distribuição geográfica das compras, fornecendo insights estratégicos para otimizar operações e melhorar a experiência do cliente.
##### **D . Quarto passo: Construção de dashboards e recomendações estratégicas:**
##### As informações extraídas serão visualizadas por meio de gráficos interativos no Power BI, permitindo uma análise mais intuitiva dos dados. A partir dessas observações, serão desenvolvidas recomendações estratégicas para otimizar a precificação, ajustar campanhas promocionais e fortalecer a fidelização de clientes, garantindo maior eficiência nas operações do marketplace.

### **Preparação e Validação dos dados**

In [2]:
import pandas as pd
import sqlite3

# Caminhos dos arquivos CSV
pedidos_csv =  r"C:\Users\Carolyne Soares\.vscode\Case- Togarantido\pedidos.csv"
produtos_csv = r"C:\Users\Carolyne Soares\.vscode\Case- Togarantido\produtos.csv"
clientes_csv = r"C:\Users\Carolyne Soares\.vscode\Case- Togarantido\clientes.csv"


# Lendo os CSVs
df_pedidos = pd.read_csv(pedidos_csv)
df_produtos = pd.read_csv(produtos_csv)
df_clientes = pd.read_csv(clientes_csv)

# Criando a conexão com um banco de dados SQLite em memória
conn = sqlite3.connect(":memory:")

# Salvando os DataFrames como tabelas no banco
df_pedidos.to_sql("pedidos", conn, index=False, if_exists="replace")
df_produtos.to_sql("produtos", conn, index=False, if_exists="replace")
df_clientes.to_sql("clientes", conn, index=False, if_exists="replace")

# Verificando se há valores ausentes ou nulos em campos importantes

queries_nulos= {
    "Clientes": "SELECT * FROM clientes WHERE cliente_id IS NULL;",
    "Pedidos": "SELECT * FROM pedidos WHERE cliente_id IS NULL OR pedido_id IS NULL;",
    "Produtos": "SELECT * FROM produtos WHERE produto_id IS NULL;"
}

for nome, query in queries_nulos.items():
    resultado = pd.read_sql_query(query, conn)
    print(f"\nValores nulos em {nome}:")
    print(resultado)


Valores nulos em Clientes:
Empty DataFrame
Columns: [cliente_id, nome_cliente, cidade, estado, data_cadastro, total_pedidos]
Index: []

Valores nulos em Pedidos:
Empty DataFrame
Columns: [pedido_id, cliente_id, data_pedido, produto_id, quantidade, preco_unitario, status_pedido, custo_produto, metodo_pagamento]
Index: []

Valores nulos em Produtos:
Empty DataFrame
Columns: [produto_id, nome_produto, categoria, fornecedor, estoque_disponivel]
Index: []


##### Não há valores nulos nas colunas cliente_id, pedido_id  e produto_id, que são muito importantes para nossa análise.

In [3]:
# Verificando se há valores duplicados nas colunas cliente_ID, pedido_ID e produto_ID.

queries_duplicados= {
    "Clientes": "SELECT cliente_id, COUNT(*) AS quantidade FROM clientes GROUP BY cliente_id HAVING COUNT(*) > 1;",
    "Pedidos": "SELECT pedido_id, COUNT(*) AS quantidade FROM pedidos GROUP BY pedido_id HAVING COUNT(*) > 1;",
    "Produtos": "SELECT produto_id, COUNT(*) AS quantidade FROM produtos GROUP BY produto_id HAVING COUNT(*) > 1;"
    }

for nome, query in queries_duplicados.items():
    duplicatas = pd.read_sql_query(query, conn)  
    print(f"Duplicatas na tabela {nome}:")
    print(duplicatas)

Duplicatas na tabela Clientes:
Empty DataFrame
Columns: [cliente_id, quantidade]
Index: []
Duplicatas na tabela Pedidos:
Empty DataFrame
Columns: [pedido_id, quantidade]
Index: []
Duplicatas na tabela Produtos:
Empty DataFrame
Columns: [produto_id, quantidade]
Index: []


##### Não há valores duplicados nas colunas mencionadas.

##### Com os dados devidamente carregados, estruturados e validados, seguiremos para a Análise Exploratória dos Dados, onde responderemos às principais questões de negócio.

### **Análise Exploratória dos Dados**

### **1. Qual foi o faturamento mensal do marketplace no período analisado?**


In [4]:
query_periodo= """
SELECT MIN(data_pedido) AS menor_data, MAX (data_pedido) AS maior_data
FROM pedidos; """

periodo_analisado = pd.read_sql_query(query_periodo, conn)
print(periodo_analisado)

   menor_data  maior_data
0  2024-02-26  2025-02-25


##### O período que estamos analisado é de 26/02/2024 a 25/02/2025.



In [5]:
query_faturamento="""
    SELECT
    strftime('%Y', data_pedido) AS ano,
    strftime('%m', data_pedido) AS mes,
    SUM(quantidade * preco_unitario) AS faturamento_total
    FROM pedidos
    WHERE status_pedido = 'Concluído'
    GROUP BY ano, mes
    ORDER BY ano, mes ;"""

faturamento_mensal = pd.read_sql_query(query_faturamento, conn)
print(faturamento_mensal)

     ano mes  faturamento_total
0   2024  02            6134.37
1   2024  03          302697.09
2   2024  04          315860.70
3   2024  05          185763.13
4   2024  06          411797.91
5   2024  07          422559.85
6   2024  08          320931.12
7   2024  09          281829.77
8   2024  10          304837.81
9   2024  11          221876.42
10  2024  12          396642.38
11  2025  01          376026.03
12  2025  02          236952.65


#####  No período analisado, julho de 2024 registrou o maior faturamento, alcançando 422.559,85 reais. Em seguida, junho de 2024 obteve o segundo maior faturamento, com 411.797,91 reais, possivelmente impulsionado por promoções de meio de ano ou pelo período de férias escolares. Já dezembro de 2024 apresentou o terceiro maior faturamento  396.642,38 reais, o que pode estar relacionado ao aumento das compras de fim de ano. Para uma análise mais detalhada, seria interessante comparar o faturamento de cada mês com o mesmo período em anos diferentes, como fevereiro de 2024 e fevereiro de 2025, a fim de identificar tendências sazonais e avaliar o crescimento ou queda das vendas ao longo do tempo.

##### Entretanto, a limitação do conjunto de dados restringe uma comparação mais ampla entre anos, já que os registros disponíveis abrangem apenas parte de 2024 e o início de 2025. Destaca-se, ainda, o crescimento expressivo em fevereiro de 2025, que saltou de 6.134,37 reais (menor valor de 2024) para 236.952,65 reais, sugerindo uma possível mudança no desempenho das vendas no início do ano.

### **2.  Qual foi o lucro mensal do marketplace no período analisado?**

In [6]:
query_lucro="""
SELECT
    strftime('%Y', data_pedido) AS ano,
    strftime('%m', data_pedido) AS mes,
    SUM((quantidade * preco_unitario) - (quantidade * custo_produto)) AS lucro
FROM pedidos
WHERE status_pedido = 'Concluído'
GROUP BY ano, mes
ORDER BY ano, mes;"""

lucro_mensal = pd.read_sql_query(query_lucro, conn)
print(lucro_mensal)

     ano mes      lucro
0   2024  02   -3864.15
1   2024  03   90086.86
2   2024  04   59489.96
3   2024  05   -4441.10
4   2024  06  109026.12
5   2024  07  161477.99
6   2024  08   72893.98
7   2024  09   22639.32
8   2024  10  -28532.93
9   2024  11   27189.01
10  2024  12   78818.32
11  2025  01   48684.40
12  2025  02   86790.47


##### Os meses de Junho e julho de 2024 foram os meses mais lucrativos, com lucros de 109.026,12 e 161.477,99 reais, respectivamente. Isso confirma que esses períodos tiveram o melhor desempenho do marketplace, alinhando-se com os meses de maior faturamento. Além disso, fevereiro de 2025 registrou um lucro expressivo de 86.790,47 reais, contrastando com o prejuízo de fevereiro de 2024 (R$ -3.864,15 ), o que pode indicar um crescimento significativo e melhorias na estratégia adotada.

### **3. Qual foi a margem de lucro média mensal considerando o custo de produtos?**

In [7]:
query_margem_lucro="""
WITH lucro_mensal AS (
    SELECT
        strftime('%Y', data_pedido) AS ano,
        strftime('%m', data_pedido) AS mes,
        SUM((quantidade * preco_unitario) - (quantidade * custo_produto)) AS lucro_bruto,
        SUM(quantidade * preco_unitario) AS receita_total
    FROM pedidos
    WHERE status_pedido = 'Concluído'
    GROUP BY ano, mes
)

SELECT
    ano,
    mes,
    ROUND((lucro_bruto / receita_total) * 100,2) AS "margem_lucro(%)"
FROM lucro_mensal
ORDER BY ano, mes;"""

margem_lucro = pd.read_sql_query(query_margem_lucro, conn)
print(margem_lucro)

     ano mes  margem_lucro(%)
0   2024  02           -62.99
1   2024  03            29.76
2   2024  04            18.83
3   2024  05            -2.39
4   2024  06            26.48
5   2024  07            38.21
6   2024  08            22.71
7   2024  09             8.03
8   2024  10            -9.36
9   2024  11            12.25
10  2024  12            19.87
11  2025  01            12.95
12  2025  02            36.63


##### A margem de lucro média mensal variou significativamente ao longo do período, com destaque para fevereiro de 2024 (-62,99%), que registrou o pior desempenho devido a custos que superaram a receita. Outros meses, como maio e outubro de 2024, também apresentaram margens negativas, possivelmente por descontos elevados ou aumento nos custos. Em contrapartida, junho e julho de 2024 tiveram os melhores resultados. Já fevereiro de 2025 mostrou uma recuperação expressiva, sugerindo melhorias na estratégia de vendas e no controle de custos.

### **4. Qual é a taxa de cancelamento mensal de pedidos?**

In [8]:
query_cancelamento_mensal= """
SELECT strftime('%Y', data_pedido) AS ano,
    strftime('%m', data_pedido) AS mes,
    ROUND(COUNT(CASE WHEN status_pedido = 'Cancelado' THEN 1 END) * 100.0 / COUNT(*),2) AS taxa_cancelamento
FROM pedidos
GROUP BY ano, mes
ORDER BY ano, mes;"""


taxa_cancelamento = pd.read_sql_query(query_cancelamento_mensal, conn)
print(taxa_cancelamento)

     ano mes  taxa_cancelamento
0   2024  02              80.00
1   2024  03              45.71
2   2024  04              50.59
3   2024  05              55.26
4   2024  06              51.65
5   2024  07              49.48
6   2024  08              56.67
7   2024  09              50.79
8   2024  10              48.00
9   2024  11              52.70
10  2024  12              47.19
11  2025  01              41.57
12  2025  02              47.89


##### **E qual seria taxa de cancelamento médio geral do periódo em análise?**

In [9]:
query_cancelamento_medio= """
SELECT
    (COUNT(CASE WHEN status_pedido = 'Cancelado' THEN 1 END) * 100.0 / COUNT(*)) AS taxa_cancelamento_geral
FROM pedidos """;

cancelamento_medio = pd.read_sql_query(query_cancelamento_medio, conn)
print(cancelamento_medio)

   taxa_cancelamento_geral
0                     49.9


##### A taxa de cancelamento se mostrou alta e instável ao longo do período analisado, com uma média de cancelamento geral de 49,9%. O maior índice foi registrado em fevereiro de 2024, atingindo 80%, o que pode indicar falhas no controle de cancelamentos, problemas operacionais ou baixa satisfação dos clientes. Esse dado se alinha ao faturamento e lucro extremamente baixos no mesmo período, sugerindo que os cancelamentos impactaram diretamente o desempenho financeiro do marketplace. Por outro lado, fevereiro de 2025 apresentou uma melhora significativa, reduzindo a taxa para 47,9%. Esse movimento acompanha o aumento do faturamento e da margem de lucro no mesmo mês, indicando possíveis ajustes na política de cancelamento ou melhorias na experiência do cliente.

### **5. Quais são os três produtos mais vendidos e seus respectivos faturamentos?**

In [10]:
query_top_3= """
SELECT p.nome_produto, SUM(pe.quantidade) AS total_vendido,
            SUM(pe.quantidade * pe.preco_unitario) AS faturamento
FROM pedidos pe
JOIN produtos p ON pe.produto_id = p.produto_id
WHERE pe.status_pedido = 'Concluído'
GROUP BY p.nome_produto
ORDER BY total_vendido DESC
LIMIT 3;"""

top_3_produtos = pd.read_sql_query(query_top_3, conn)
print(top_3_produtos)

   nome_produto  total_vendido  faturamento
0  Produto 2034             61    167730.42
1  Produto 2019             56    163951.82
2  Produto 2014             52    157427.16


##### O Produto 2034 foi o mais vendido, com 61 unidades, gerando um faturamento de 167.730,42 reais . O Produto 2019 ficou em segundo lugar, com 56 unidades vendidas, faturando 163.951,82 reais. O Produto 2014 foi o terceiro mais vendido, com 52 unidades, faturando 157.427,16 reais. **Mas apesar de serem esses os top 3 mais vendidos, será que foram esses produtos que deram maiores lucros para empresa?**

In [11]:
query_margem_lucro_por_produto= """
SELECT
    p.nome_produto,
    SUM((pe.quantidade * pe.preco_unitario) - (pe.quantidade * pe.custo_produto)) AS lucro_total,
    ROUND(SUM((pe.quantidade * pe.preco_unitario) - (pe.quantidade * pe.custo_produto)) * 100.0 / SUM(pe.quantidade * pe.preco_unitario), 2) AS margem_lucro_percentual
FROM pedidos pe
JOIN produtos p ON pe.produto_id = p.produto_id
WHERE pe.status_pedido = 'Concluído'
GROUP BY p.nome_produto
ORDER BY lucro_total DESC
LIMIT 3; """

margem_lucro_por_produto=pd.read_sql_query(query_margem_lucro_por_produto, conn)
print(margem_lucro_por_produto)


   nome_produto  lucro_total  margem_lucro_percentual
0  Produto 2014     60054.74                    38.15
1  Produto 2034     59081.55                    35.22
2  Produto 2035     48902.84                    45.19


#####  Assim, apesar dos produtos mais vendidos serem o Produto 2034, Produto 2019 e Produto 2014, quando analisamos o lucro total, o Produto 2014 lidera com 60.054,74 reais, seguido pelo Produto 2034 (59.081,55 reais) e Produto 2035 (48.902,84 reais). Isso significa que o Produto 2019, apesar de ser o segundo mais vendido, não aparece entre os três mais lucrativos, possivelmente devido a uma margem de lucro menor. Já o Produto 2035, que nem estava entre os mais vendidos, aparece no top 3 produtos mais lucrativos, mostrando que uma boa margem de lucro pode ser mais vantajosa do que apenas um alto volume de vendas.

### **6. Quais são os métodos de pagamento mais utilizados?**




In [12]:
query_metodo_pagamento= """
SELECT metodo_pagamento, COUNT(*) AS total_uso
FROM pedidos
WHERE status_pedido = 'Concluído'
GROUP BY metodo_pagamento
ORDER BY total_uso DESC;"""

metodo_pagamento=pd.read_sql_query(query_metodo_pagamento, conn)
print(metodo_pagamento)

         metodo_pagamento  total_uso
0  Transferência Bancária        137
1                     PIX        124
2       Cartão de Crédito        123
3                  Boleto        117


##### A análise dos métodos de pagamento mais utilizados revela que a Transferência Bancária foi a opção mais escolhida pelos clientes, com 137 transações, seguida de perto pelo PIX (124 transações) e Cartão de Crédito (123 transações). O Boleto Bancário, apesar de ser o menos utilizado entre os quatro, ainda teve um número significativo de transações (117).

### **7. Quais estados geraram mais vendas?**

In [13]:
# Para responder a essa pergunta eu levei em consideração a quantidade total de Pedido_id gerado por Estado com status de concluído.

query_por_estado= """
SELECT c.estado, COUNT(p.pedido_id) AS total_vendas
FROM pedidos p
JOIN clientes c ON p.cliente_id = c.cliente_id
WHERE p.status_pedido = 'Concluído'
GROUP BY c.estado
ORDER BY total_vendas DESC
LIMIT 4;"""

vendas_por_estado=pd.read_sql_query(query_por_estado, conn)
print(vendas_por_estado)

  estado  total_vendas
0     MT            21
1     SC            19
2     HI            16
3     CT            16


##### A análise das vendas por estado mostra que MT lidera com 21 pedidos concluídos, seguido por SC com 19 vendas e HI e CT com 16 vendas cada. **Mas será que esses Estado estão entre aqueles que geraram maiores lucros para empresa?**

In [14]:
query_por_estado_lucro= """
SELECT
    c.estado,
    SUM((p.quantidade * p.preco_unitario) - (p.quantidade * p.custo_produto)) AS lucro_total
FROM pedidos p
JOIN clientes c ON p.cliente_id = c.cliente_id
WHERE p.status_pedido = 'Concluído'
GROUP BY c.estado
ORDER BY lucro_total DESC
LIMIT 4;"""

lucro_por_estado=pd.read_sql_query(query_por_estado_lucro, conn)
print(lucro_por_estado)

  estado  lucro_total
0     MT     89426.18
1     MD     63833.29
2     WI     44216.18
3     MN     41728.85


##### Observa-se que estado MT não apenas liderou em número de pedidos concluídos (21), mas também registrou o maior lucro total (R$ 89.426,18). Isso indica que as vendas no estado não apenas foram volumosas, mas também rentáveis. Já o estado SC, que estava em segundo lugar em número de vendas (19), não apareceu no ranking dos estados mais lucrativos. Isso pode indicar que, apesar de ter muitas vendas, o ticket médio ou a margem de lucro dos produtos vendidos em SC pode ser menor. Estados como MD, WI e MN aparecem com alto lucro, mas não necessariamente tinham os maiores volumes de pedidos. Isso pode significar que esses estados compraram produtos mais caros ou com maior margem de lucro.

### **8. Quantos clientes realizaram mais de uma compra?**

In [15]:
query_clientes_multipla_compra= """
SELECT COUNT(*) AS clientes_recorrentes
FROM (
        SELECT cliente_id
        FROM pedidos
        WHERE status_pedido = 'Concluído'
        GROUP BY cliente_id
        HAVING COUNT(pedido_id) > 1);"""

clientes_multipla_compra=pd.read_sql_query(query_clientes_multipla_compra, conn)
print(clientes_multipla_compra)

   clientes_recorrentes
0                   130


##### A consulta revelou que 130 clientes realizaram mais de uma compra no marketplace.Contudo, é interessante comparar esse número com os clientes que compraram apenas uma vez.

In [16]:
query_clientes_uma_compra= """
SELECT COUNT(*) AS clientes_nao_recorrentes
FROM (
        SELECT cliente_id
        FROM pedidos
        WHERE status_pedido = 'Concluído'
        GROUP BY cliente_id
        HAVING COUNT(pedido_id) = 1);"""

clientes_uma_compra=pd.read_sql_query(query_clientes_uma_compra, conn)
print(clientes_uma_compra)

   clientes_nao_recorrentes
0                       190


##### Assim, embora tenhamos 130 clientes recorrentes, ou seja, que efetuaram mais de uma compra, há 190 clientes que não são recorrentes, tendo feito apenas uma compra no marketplace. Isso indica que, apesar de ter uma parcela considerável de clientes que retornam para novas compras, a maioria ainda não se fidelizou. Para melhorar essa retenção, seria interessante adotar estratégias como ofertas personalizadas, descontos progressivos e programas de fidelidade, incentivando os clientes não recorrentes a realizarem novas compras. Como próximo passo, vamos analisar o ticket médio desses clientes e identificar quais produtos são mais adquiridos por eles.

In [17]:
query_ticket_medio_recorrentes = """
SELECT
    SUM(pe.quantidade * pe.preco_unitario) / COUNT(DISTINCT pe.pedido_id) AS ticket_medio
FROM pedidos pe
WHERE pe.cliente_id IN (
    SELECT cliente_id
    FROM pedidos
    WHERE status_pedido = 'Concluído'
    GROUP BY cliente_id
    HAVING COUNT(pedido_id) > 1
);"""

ticket_medio_recorrentes=pd.read_sql_query(query_ticket_medio_recorrentes, conn)
print(ticket_medio_recorrentes)

   ticket_medio
0   7462.148318


##### Dessa forma, o ticket médio dos clientes recorrentes é de aproximadamente R$ 7.462,15. Isso quer dizer que, em média, cada pedido feito por um cliente que realizou mais de uma compra possui um valor elevado. Esse resultado reforça a importância da fidelização de clientes, pois aqueles que voltam a comprar tendem a gastar mais por pedido.

In [18]:
query_produtos_recorrentes = """
WITH clientes_recorrentes AS (
    SELECT cliente_id
    FROM pedidos
    WHERE status_pedido = 'Concluído'
    GROUP BY cliente_id
    HAVING COUNT(pedido_id) > 1
)

SELECT
    p.nome_produto,
    SUM(pe.quantidade) AS total_vendido
FROM pedidos pe
JOIN produtos p ON pe.produto_id = p.produto_id
WHERE pe.cliente_id IN (SELECT cliente_id FROM clientes_recorrentes)
GROUP BY p.nome_produto
ORDER BY total_vendido DESC
LIMIT 4;"""


produtos_recorrentes=pd.read_sql_query(query_produtos_recorrentes, conn)
print(produtos_recorrentes)

# Fechando a conexão
conn.close()

   nome_produto  total_vendido
0  Produto 2019             56
1  Produto 2004             43
2  Produto 2026             42
3  Produto 2013             42


##### A análise mostra que o Produto 2019 é o mais comprado por clientes recorrentes, com 56 unidades vendidas, seguido pelos Produtos 2004, 2026 e 2013, todos com um volume de vendas entre 42 e 43 unidades. Curiosamente, o Produto 2019, que não figurava entre os três mais lucrativos no total, destaca-se como o mais comprado por clientes fiéis. Isso sugere que ele pode ser um item essencial ou de alto engajamento, possivelmente impulsionado por sua utilidade ou recorrência de compra. Os produtos dessa lista são ótimos candidatos para estratégias de recompra e promoções direcionadas a clientes recorrentes.

### **Considerações Finais**

##### As consultas SQL realizadas permitiram obter insights estratégicos sobre o desempenho do marketplace, abordando desde o faturamento e lucro até o comportamento dos clientes e os métodos de pagamento mais utilizados. A análise revelou tendências importantes, como sazonalidade nas vendas, impacto da fidelização de clientes e oportunidades de otimização na precificação e na redução da taxa de cancelamento.

##### Vale destacar que, apesar de termos identificado padrões relevantes, há espaço para análises mais aprofundadas, como a correlação entre produtos de alto faturamento e suas margens de lucro, a influência dos métodos de pagamento na taxa de cancelamento e possíveis variações regionais nas vendas. Além disso, a análise preditiva com base no comportamento passado poderia auxiliar na antecipação de picos de demanda e estratégias de precificação mais eficazes.


##### O relatório completo, tomando como base essas consultas, está disponível em PDF.
